In [1]:
import re
import openai
import os
import inspect
import pyperclip
import pyautogui
from colored import fg, attr
import logging
import black

from pygments import highlight
from pygments.lexers import PythonLexer
from pygments.formatters import TerminalFormatter

openai.api_key = os.getenv("OPENAI_API_KEY")

In [2]:
%load_ext jupyter_ai_magics

In [33]:
def format_py_code(code):
    """Return a formatted version of Python code."""
    try:
        formatted_code = black.format_str(code, mode=black.FileMode())
    except Exception:
        formatted_code = code
    
    return formatted_code

In [71]:
def format_python_code(code):
    """ This function will format python code in the terminal. 
    Args:
        code: a string, the python code to format
    Returns:
        formatted_code: a string, the formatted code
    """
    # Split the code into lines
    lines = code.split('\n')
    # Initialize the list of formatted lines
    formatted_lines = []
    # Loop through each line of code
    for line in lines:
        # Format the line
        formatted_line = highlight(line, PythonLexer(), TerminalFormatter())
        # Add the formatted line to the list
        formatted_lines.append(formatted_line)
    # Join the lines together
    formatted_code = '\n'.join(formatted_lines)
    
    return formatted_code


In [72]:

def show_function_definition(function):
    """Return the source code for a function."""
    
    try:
        source_lines, _ = inspect.getsourcelines(function)
    except TypeError:
        return "This function does not have a definition."
    except OSError:
        return "This function does not have a definition."
    
    # Join the lines together into a single string.
    function_definition = ''.join(source_lines)
    
    # Format the code using the format_python_code function.
    formatted_definition = format_py_code(function_definition)
    
    return formatted_definition


In [73]:
def extract_function_name(text):
    # Find the first word after 'def' and before the first '(', if any.
    match = re.search(r'(?:def|)(\w+)\(', text)
    if match:
        # The match object always has a 'group' method.
        function_name = match.group(1)
        return function_name
    else:
        # If there is no match, return an empty string.
        return ''


In [74]:

def extract_function_name_from_stacktrace(text):
    
    # Find the index of " in " and add 4 to get the start index
    start_index = text.find(" in " ) + 4
    
    # Find the index of the next newline character
    end_index = text.find("\n", start_index)
    
    # Extract the text between the two indices, and strip whitespace
    extracted_text = text[start_index:end_index].strip()
    
    # Return the extracted text
    return extract_function_name(extracted_text)



In [75]:

def split_string_last_cell_in(input_string):
    # Find the last instance of "Cell In" in the string
    index = input_string.rfind("Cell In")
    if index != -1:
        # If there is an instance of "Cell In" in the string,
        # then extract the function name from the stack trace
        stack_trace = input_string[index:].strip()
        func_name = extract_function_name_from_stacktrace(stack_trace)
        if func_name:
            return func_name
        else:
            return ''
    else:
        # If there is no instance of "Cell In" in the string,
        # then return an empty string
        return ""


In [76]:

def get_error_function():
    """Returns the source code of the last error function in the Err dictionary."""
    
    # Get the name of the last function in the Err dictionary.
    last_function_name = split_string_last_cell_in(Err[list(Err.keys())[-1]])
    
    # If the function exists in the global namespace and is callable, get its source code.
    if last_function_name in globals() and callable(globals()[last_function_name]):
        last_function = globals()[last_function_name]
        function_definition = inspect.getsource(last_function)
        return function_definition
    
    # If the function doesn't exist, print an error message and return an empty string.
    else:
        print("A function does not exist.")
        return ''


In [77]:
def explain_error( model = "text-davinci-003"):
    
    # loop through all errors and get the last one
    try:
        err_index = list(Err.keys())[-1]
    except NameError:
        print('Everything is alright, no error to fix')
        return 
    except Exception as e:
        print(e)
        return
    else:
        # get the error_code that produced the error
        error_code = get_error_function() or In[err_index]
        # get the number of characters in the previous error_code
        char_no = 850 + len(Err[err_index]) + len(In[err_index]) # 850 is the length of prompt
        # get the number of characters remaining
        rest_char_no = 4000 - char_no
        # create the prompt message
        prompt = f'Error \"{Err[err_index][-rest_char_no:]} \" was produced from following error_code \"{error_code} \" .Explain the error. Rewrite error_code! Wrap your rewritten code in 5 brackets like this <<<<<  >>>>>, 5 ok! ',
        response = openai.Completion.create(
                  model=model,
                  prompt=prompt,
                  temperature=0,
                  max_tokens=182,
                  top_p=1.0,
                  frequency_penalty=0.0,
                  presence_penalty=0.0,
                  stop=["###"]
                )    
        return response['choices'][0]['text']

In [78]:
def invoke_magic(magic_name, magic_arguments, code):
    """
    Invoke a line or cell magic in Jupyter Notebook.

    Args:
        magic_name (str): Name of the magic, including the leading '%' character.
        magic_arguments (str): Arguments to pass to the magic.
        code (str): Code to execute within the magic.
    """
    if magic_name.startswith("%%"):
       
        return get_ipython().run_cell_magic(magic_name[2:], magic_arguments, code)
    elif magic_name.startswith("%"):
        
        return get_ipython().run_line_magic(magic_name[1:], magic_arguments)
    else:
        raise ValueError("Invalid magic name. Must start with '%%' for cell magic or '%' for line magic.")

def explain_error_with_chatgpt():
    
    func = show_function_definition(func) or In[-2]
        
    err_index = list(Err.keys())[-1]
    revised_code = invoke_magic('%%ai','chatgpt',f'Following code \" {func} \" produces error \" {Err[err_index]} \".\
                                 do not Explain the error! Write correct code! Wrap your code in <<< your code >>>' )
    
    return revised_code
            

In [79]:

def revise_code(explanation):
    """ This code checks to see if the explanation is valid and extracts the revised code from it.
    
    Args:
        explanation: a string, the explanation text
    
    Returns:
        revised_code: a string, the revised code or an empty string
    """
    
    if explanation:
        print("Explanation:\n", explanation )
        # Split the explanation at the start and end of the revised code
        split_explanation = explanation.split("<<<<<")#" your code >>>>>")
        # Ensure that the explanation has the expected format
        if len(split_explanation) >= 2:
            # Extract and return the revised code
            revised_code = split_explanation[1].split(">>>>>")[0]
            return revised_code
        else:
            print("Invalid explanation format.")
    else:
        print("No explanation provided.")
    
    return ""  # Return an empty string if no revised code is available



In [80]:
# This code will emulate the user typing the command "esc b" then pressing enter.
# Next, the code will emulate the user typing the command "command v" which will paste the previously copied text.

def emulate_typing():
    # Presses esc then b, then enter
    pyautogui.hotkey('esc','b')
    pyautogui.press('enter')

    # Presses command then v
    pyautogui.hotkey('command', 'v')


In [ ]:
def code_error_fixer():
    """ This code will get the error produced by the last line of code and then attempt to fix it."""
    
    # Get the error and return an explanation
    explained_error = explain_error()
    logging.info('explained_error: {}'.format(explained_error))
    
    # If there is an error, try to fix it
    if explained_error:
        # Return the revised code
        revised_code = revise_code(explained_error)
        logging.info('revised_code: {}'.format(revised_code))
        # If the revised code exists, format it and copy it to the clipboard
        if revised_code:
            print(format_python_code(revised_code))
            pyperclip.copy(revised_code)
            # Ask the user if they want to apply the revised code
            your_choice = input('Apply code? Press Enter to apply code or any other key >> Enter to cancel')
            if not your_choice:
                emulate_typing()
            else:
                pass
        else:
            print('No suggestion')
    else:
        pass
    

In [93]:
def copy_to_clipboard():
    # try to copy the last error message in the Err dict to the clipboard
    try:
        pyperclip.copy(Err[list(Err.keys())[-1]])
    except Exception:
        pass
copy_to_clipboard()

In [92]:
code_error_fixer()

NameError: name 'Exceptio' is not defined

In [ ]:
Err[list(Err.key())[-1]]

In [94]:
copy_to_clipboard()